# Imports

In [39]:
import pandas as pd
import numpy as np
import itertools
from math import pi
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [ ]:
df_customers = pd.read_pickle('../../data/customers.pkl')
df_customers.info()

## RFM

To start our segmentation, we will simply follow the RFM marketing method that consists in categorize customers according 3 main features :
- recency : when was the last order
- frequency : how often customers order
- monetary : how much a costumor can spend

In [ ]:
df_rfm = df_customers[
    [
        "customer_unique_id",
        "days_since_last_order", # recency
        "total_orders", # frequency
        "total_expenses", # monetary
    ]
]

df_rfm.info()

In [42]:
# replace 0 days by 1 day to avoid having 0
df_rfm['days_since_last_order'] = df_rfm["days_since_last_order"].apply(lambda x: 1 if x == 0 else x)

## Scoring (The idea was to libellised groups but we did not continue this way)

#### Recency score

In [ ]:
# See the last order date distribution showing cde
plt.figure(figsize=(20, 6))
sns.displot(df_rfm["days_since_last_order"], kde=True)
plt.axvline(
    df_rfm["days_since_last_order"].mean(), color="red", linestyle="--", label="Mean"
)
plt.axvline(
    df_rfm["days_since_last_order"].median(),
    color="green",
    linestyle="--",
    label="Median",
)
plt.title('Days Since Last Order Distribution')
plt.xlabel('Days Since Last Order')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# Boxplot the distribution of days since last order
fig = px.box(df_rfm, x="days_since_last_order", title='Distribution of Days Since Last Order', width=800, height=400)

fig.show()

In [45]:
# create a new column that categorizes the days since last order according to the quantiles.

# Define the quantiles
quantiles = df_rfm['days_since_last_order'].quantile(q=[0.25, 0.5, 0.75, 1])

# Create a function that assigns a category to each customer based on the quantiles
def categorize_days_since_last_order(days):
    if days <= quantiles[0.25]:
        return 5
    elif days <= quantiles[0.5]:
        return 4
    elif days <= quantiles[0.75]:
        return 3
    elif days <= quantiles[1]:
        return 2
    else:
        return 1

# Apply the function to the days_since_last_order column
df_rfm['recency_score'] = df_rfm['days_since_last_order'].apply(categorize_days_since_last_order)

#### Frequency score

In [ ]:
# Plot the histogram of total orders
fig =  px.histogram(df_rfm, x='total_orders', title='Distribution of Recency Score')
fig.show()

In [ ]:
# Count each unique value in the total_orders columns, add the % of customers that fall into each category
df_rfm['total_orders'].value_counts(normalize=True) * 100


97% of customers did a unique order. Calculate a scoreon the frequencies does not give us a valuable information.

Customers order at least one time and 16 times at the most. How is it distributed ?

#### Monetary

In [ ]:
df_rfm["total_expenses"] = df_rfm["total_orders_value"] + df_rfm["total_freight_value"]

In [ ]:
# Plot the total expenses distribution
group_labels = ["Order value", "Freight value"]

fig = ff.create_distplot([df_rfm["total_orders_value"], df_rfm["total_freight_value"]], group_labels, bin_size=1000, curve_type='kde')

# Overlay histograms
fig.update_layout(title='Distribution of Total Expenses', barmode='overlay')
fig.show()

In [ ]:
# boxplot the distribution of total expenses
fig = px.box(df_rfm, x='total_expenses', title='Distribution of Total Expenses')
fig.show()

In [ ]:
Q1 = np.quantile(df_rfm["total_expenses"], 0.25)
Q3 = np.quantile(df_rfm["total_expenses"], 0.75)
IQR = Q3 - Q1
expenses_distribution = df_rfm[~(df_rfm["total_expenses"] > Q3 + (1.5 * IQR))]
expenses_distribution.head()

In [ ]:
fig = px.histogram(expenses_distribution, x='total_expenses', title='Distribution of Total Expenses', width=800, height=400, marginal='box')
fig.show()

In [ ]:
# Create a monetary score column
quantiles = df_rfm['total_expenses'].quantile(q=[0.25, 0.5, 0.75, 1])

def categorize_total_expenses(expenses):
    if expenses <= quantiles[0.25]:
        return 1
    elif expenses <= quantiles[0.5]:
        return 2
    elif expenses <= quantiles[0.75]:
        return 3
    elif expenses <= quantiles[1]:
        return 4
    else:
        return 5

df_rfm['monetary_score'] = df_rfm['total_expenses'].apply(categorize_total_expenses)

In [ ]:
df_rfm.head()

Let's work on the features thats belongs to RFM :

In [ ]:
main_rfms = df_rfm[["customer_unique_id","days_since_last_order",'recency_score', 'total_orders', 'monetary_score', "total_expenses"]]

We took in consideration 2 scores : Recency and monetary. 
Let's create a RFM_score

In [ ]:
main_rfms["rfm_score"] = (main_rfms["recency_score"] + main_rfms["monetary_score"]) / 2

In [ ]:
main_rfms.head()

In [ ]:
# Plot a scatter plot of expenses per customer
fig = px.scatter(
    main_rfms,
    y='days_since_last_order',
    x='total_expenses',
    color='rfm_score',
    title='3D Scatter Plot of Expenses per Customer',
    labels={'total_orders_value': 'Total Orders Value', 'total_freight_value': 'Total Freight Value', 'total_expenses': 'Total Expenses'}
)
fig.show()

# Function utils

## Plot results

In [7]:
# Plot distribution
def scale_data_and_plot_distributions(original_df: pd.DataFrame, features: list[str], plot: bool = True):
    # Apply logarithm transformation
    df = original_df[features].copy()
    log_df = df.copy()
    log_df = df.apply(np.log1p)

    fig, axes = plt.subplots(len(features), 3, figsize=(20, 12))

    if plot:
        for i, feature in enumerate(original_df.columns):
            sns.histplot(original_df[features], kde=True, ax=axes[i, 0])
            axes[i, 0].set_title(f"{feature} original distribution")
            axes[i, 0].set_xlabel("Value")
            axes[i, 0].set_ylabel("Count")

            sns.histplot(log_df[feature], kde=True, ax=axes[i, 1])
            axes[i, 1].set_title(f"{feature} log distribution")
            axes[i, 1].set_xlabel("Value")
            axes[i, 1].set_ylabel("Count")


        plt.tight_layout()
        plt.show()

    return df, log_df

In [8]:
def plot_3d_clusters(df: pd.DataFrame, algorithm: str="KMeans"):
    """Plots a 3D scatter plot of the RFM clusters.

    Args:
        df (pd.DataFrame): Scalerd dataframe with cluster labels
    """

    fig = go.Figure()

    # Add traces for each cluster
    for cluster in sorted(df["cluster"].unique()):
        cluster_data = df[df["cluster"] == cluster]
        fig.add_trace(
            go.Scatter3d(
                x=cluster_data[cluster_data.columns[0]],
                y=cluster_data[cluster_data.columns[1]],
                z=cluster_data[cluster_data.columns[2]],
                mode="markers",
                marker=dict(size=4),
                name=f"Cluster {cluster}",
                text=cluster_data.index,  # Add index as text for annotation
                hoverinfo="text+x+y+z",
            )
        )

    fig.update_layout(
        title=f"Clusters 3D representation for {algorithm} with {len(df["cluster"].unique())} clusters and {len(df.columns.to_list())-1} features",
        scene=dict(
            xaxis_title=df.columns[0],
            yaxis_title=df.columns[1],
            zaxis_title=df.columns[2],
        ),
        legend=dict(title="Clusters"),
    )

    fig.show()


In [9]:
# Plot a radar chart of the clusters
def plot_radar_chart(df: pd.DataFrame):
    """Plots a radar chart of the clusters.

    Args:
        df (pd.DataFrame): Scaled dataframe
    """
    # Calculate the mean of each feature for each cluster
    cluster_means = df.groupby("cluster").mean()

    # Normalize the cluster means
    scaler = StandardScaler()
    cluster_means_normalized = pd.DataFrame(
        scaler.fit_transform(cluster_means), columns=cluster_means.columns
    )

    # Number of variables we're plotting
    categories = list(cluster_means_normalized.columns)
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variables)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Initialise the spider plot
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

    # Loop through each cluster and plot
    for i, row in cluster_means_normalized.iterrows():
        values = row.tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=f'Cluster {i}')
        ax.fill(angles, values, alpha=0.25)

    # Add labels
    plt.xticks(angles[:-1], categories, color="grey", size=8)

    # Add a title and legend
    plt.title("Radar Chart for Clusters", size=20, color="black", y=1.1)
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))


    fig.show()

## Clustering benchmark

In [10]:
def benchmark_clusters(original_df: pd.DataFrame, add_cols: list[str]=None) -> pd.DataFrame:
    cols = ["days_since_last_order", "total_orders", "total_expenses"]
    if add_cols:
        cols += add_cols

    # original_df = original[cols].copy()
    # original_df["cluster"] = df["cluster"]

    agg_functions = {
        "cluster":["count", lambda x : round(x.count() / original_df.shape[0] * 100, 2)],
        "days_since_last_order": lambda x: round(x.mean(),2),
        "total_orders": "mean",
        "total_expenses": [lambda x: round(x.mean(), 2), "sum", lambda x: round((x.sum() / (original_df["total_expenses"].sum())) * 100, 2)]
    }

    clusters_summary = original_df.groupby("cluster").agg(agg_functions).reset_index()

    # Flatten the MultiIndex columns
    clusters_summary.columns = [
        "_".join(col).strip() if isinstance(col, tuple) else col
        for col in clusters_summary.columns.values
    ]

    # Rename the columns
    clusters_summary = clusters_summary.rename(
        columns={
            "clusters_" : "cluster",
            "cluster_count": "customers",
            "cluster_<lambda_0>": "customers(%)",
            "total_orders_mean": "average_orders",
            "total_expenses_<lambda_0>": "average_basket(real)",
            "total_expenses_sum": "total_revenues(real)",
            "total_expenses_<lambda_1>": "total_revenues(%)",
            "days_since_last_order_<lambda>": "average_recency",
        }
    )

    display(clusters_summary)

## DBSCAN grid search

In [11]:
def grid_search(combinations, df):
    scores = []
    all_labels = []

    for i, (eps, min_samples) in enumerate(combinations):
        dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(df)
        labels = dbscan.labels_
        labels_set = set(labels)
        num_clusters = len(labels_set)
        if -1 in labels_set:
            num_clusters -= 1
        all_labels.append(labels)
        if (num_clusters < 2) or (num_clusters > 10):
            scores.append(-20)
            all_labels.append("Poor")
            print(f"Combination {i} - eps: {eps}, min_samples: {min_samples}, number of clusters: {num_clusters}")
            continue
        score = silhouette_score(df, labels)

        scores.append(score)
        all_labels.append(labels)
        print(f"Combination {i} - eps: {eps}, min_samples: {min_samples}, number of clusters: {num_clusters}, score: {score}")

    best_index = np.argmax(scores)
    best_params = combinations[best_index]
    best_labels = all_labels[best_index]
    best_score = scores[best_index]

    return {"epsylon": best_params[0], "min_samples": best_params[1], "score": best_score, "labels": best_labels}

## AHC gridsearch

In [12]:
def get_ahc_best_params(sample: pd.DataFrame, distance_measures: list[str], linkage_methods: list[str]):
    """Gets the best params for the CHA method.

    Args:
        sample (pd.DataFrame): dataframe to be used
        distance_measures (list[str]):  list of distances between individuals
        linkage_methods (list[str]): List of methods to calculate distances between clusters
    """
    # Find optimal distances between individuals and clusters
    scores_dict = {}

    # Iterate over the distance measures to find the best silhouette score
    for distance in distance_measures:
        for linkage_method in linkage_methods:
            # Create the linkage matrix
            if (linkage_method == "ward" or linkage_method == "centroid") and distance != "euclidean":
                # distance = "euclidean"
                # Z = linkage(sample, method=linkage_method, metric=distance)
                continue
            # else:
            Z = linkage(sample, method=linkage_method, metric=distance)

            # Set a distance threshold
            # max_d =
            # clusters = fcluster(Z, max_d, criterion="distance")
            max_clusters = 10

            best_score = -1


            max_clusters = 10

            for t in np.linspace(0, np.max(Z[:, 2]), 100):
                clusters = fcluster(Z, t, criterion="distance")
                if len(set(clusters)) > 1 and len(set(clusters)) <= max_clusters:
                    score = silhouette_score(sample, clusters)
                    if score > best_score:
                        best_score = score
                        best_threshold = t

            scores_dict[(distance, linkage_method)] = best_score
            print(f"Distance: {distance}, Linkage: {linkage_method}, Score: {best_score}")
                        # # Calculate the silhouette score
                        # if len(set(clusters)) > 1:
                        #     average_score = silhouette_score(sample, clusters, metric=distance)
                        #     scores_dict[(distance, linkage_method)] = average_score
                        #     print(f"Distance: {distance}, Linkage: {linkage_method}, Score: {average_score}")

    # Get the best distance measure and linkage method
    best_params = max(scores_dict, key=scores_dict.get)
    print(f"Best parameters: {best_params}")
    return best_params

# Data preparation

In [ ]:
# Plot the df_rfm distribution for each feature
fig, axes = plt.subplots(3, 1, figsize=(20, 12))

for i, feature in enumerate(df_rfm.columns[1:]):
    sns.histplot(df_rfm[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'{feature} Distribution')
    plt.tight_layout()

plt.show()

### Scaling data

### Standard Scaler

For the current working dataset, the selected features are numericals. However, we are dealing with numbers of orders, number of days and expenses amount, which means different scales. To start, we will use the StandardScaler algorithm that will transform data so that it has a mean of 0 and a standard deviation of 1.

In [ ]:
# df_rfm.set_index('customer_unique_id', inplace=True)
df_rfm.drop(columns=["customer_unique_id"], inplace=True)

# Initialize sclaler
scaler = StandardScaler()

# Fit and transform the data
scaled_features = scaler.fit_transform(df_rfm)

# Create a dataframe from the scaled features
scaled_df_rfm = pd.DataFrame(scaled_features, columns=df_rfm.columns)

scaled_df_rfm.head()

### MinMax Scaler

In [ ]:
# initialize scaler
minmax_scaler = MinMaxScaler()
minmax_scaled_features = minmax_scaler.fit_transform(df_rfm)

# Create a dataframe from the scaled features
minmax_scaled_df_rfm = pd.DataFrame(minmax_scaled_features, columns=df_rfm.columns)

minmax_scaled_df_rfm.head()

### Log before normalization

We also will try to apply the logarithm before scaling as the distribution can be asymetrical.

In [ ]:
zero_values = (df_rfm == 0).sum()
neg_values = (df_rfm < 0).sum()
print("Columns with zero values:\n", zero_values)
print("\nColumns with negative values:\n", neg_values)

# We can proceed with logarithm transformation on days_since_last_order and total_expenses
log_df_rfm = np.log1p(df_rfm)

# Add the remaining columns to the log_df_rfm dataframe
# log_df_rfm["total_orders"] = df_rfm["total_orders"]

# Check for NaN values after applying log1p
nan_values = log_df_rfm.isna().sum()
print("\nColumns with NaN values after log1p:\n", nan_values)

After applying the logarithm, let's normalize the features.

In [ ]:
# Compare distribution of the scaled features
fig, axes = plt.subplots(3, 4, figsize=(20, 12))

for i, feature in enumerate(df_rfm.columns):
    sns.histplot(df_rfm[feature], kde=True, ax=axes[i,0])
    axes[i,0].set_title(f'{feature} original distribution')
    axes[i,0].set_xlabel('Value')
    axes[i,0].set_ylabel('Count')

    # Scaled features
    sns.histplot(scaled_df_rfm[feature], kde=True, ax=axes[i,1])
    axes[i,1].set_title(f'{feature} scaled distribution')
    axes[i,1].set_xlabel('Value')
    axes[i,1].set_ylabel('Count')

    # MinMax scaled features
    sns.histplot(minmax_scaled_df_rfm[feature], kde=True, ax=axes[i,2])
    axes[i,2].set_title(f'{feature} MinMax scaled distribution')
    axes[i,2].set_xlabel('Value')
    axes[i,2].set_ylabel('Count')

    # Log distribution
    sns.histplot(log_df_rfm[feature], kde=True, ax=axes[i,3])
    axes[i,3].set_title(f'log_{feature} distribution')
    axes[i,3].set_xlabel('Value')
    axes[i,3].set_ylabel('Count')

plt.tight_layout()
plt.show()

### Log + normalizers

In [52]:
# Standard scaler
scaler = StandardScaler()
scaled_log_features = scaler.fit_transform(log_df_rfm)

scaled_log_df_rfm = pd.DataFrame(
    scaled_log_features, columns=df_rfm.columns
)


# MinMax scaler
minmax_scaler = MinMaxScaler()
minmax_scaled_log_features = minmax_scaler.fit_transform(log_df_rfm)

minmax_scaled_log_df_rfm = pd.DataFrame(
    minmax_scaled_log_features, columns=df_rfm.columns
)

In [ ]:
# Compare distribution of the scaled features
fig, axes = plt.subplots(3, 3, figsize=(20, 12))

for i, feature in enumerate(df_rfm.columns):
    sns.histplot(df_rfm[feature], kde=True, ax=axes[i, 0])
    axes[i, 0].set_title(f"{feature} original distribution")
    axes[i, 0].set_xlabel("Value")
    axes[i, 0].set_ylabel("Count")

    # Scaled features
    sns.histplot(scaled_log_df_rfm[feature], kde=True, ax=axes[i, 1])
    axes[i, 1].set_title(f"{feature} scaled log distribution")
    axes[i, 1].set_xlabel("Value")
    axes[i, 1].set_ylabel("Count")

    # MinMax scaled features
    sns.histplot(minmax_scaled_log_df_rfm[feature], kde=True, ax=axes[i, 2])
    axes[i, 2].set_title(f"{feature} MinMax scaled log distribution")
    axes[i, 2].set_xlabel("Value")
    axes[i, 2].set_ylabel("Count")

plt.tight_layout()
plt.show()


For our project, we will choose to continue with the standard scaler normalizer. As we can observe, the distribution is gaussian and knowing that Kmeans algorithm is based on euclidian distance, Standard Scaler seems to be the most appropriate normalization after a logarithm transformation of the data.

## Evaluate the number of clusters

In [ ]:
scaled_df_rfm.info()

In [ ]:
log_df_rfm.info()

### StandardScaler

The distorsion is the sum of squared distances from each point to its assigned center.

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2, 10), metric='distortion', timings=True)

visualizer.fit(scaled_df_rfm)
visualizer.poof()

### Log

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2, 10))

visualizer.fit(log_df_rfm)
visualizer.poof()

### Log + Normalizers

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2, 10))

visualizer.fit(scaled_log_df_rfm)
visualizer.poof()

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(2, 10))

visualizer.fit(minmax_scaled_log_df_rfm)
visualizer.poof()


## Silhouette Visualizer

Used to evaluate the density and separation between clusters

### StandardScaler

In [ ]:
for n_clusters in range(3, 5):
    model = KMeans(n_clusters=n_clusters)
    visualizer = SilhouetteVisualizer(model)

    visualizer.fit(scaled_df_rfm)
    visualizer.poof()

### Scaled log

In [ ]:
for n_clusters in range(3, 7):
    model = KMeans(n_clusters=n_clusters)
    visualizer = SilhouetteVisualizer(model)

    visualizer.fit(scaled_log_df_rfm)
    visualizer.poof()


In [ ]:
for n_clusters in range(3, 7):
    model = KMeans(n_clusters=n_clusters)
    visualizer = SilhouetteVisualizer(model)

    visualizer.fit(minmax_scaled_log_df_rfm)
    visualizer.poof()


### Log

In [ ]:
for n_clusters in range(3, 6):
    model = KMeans(n_clusters=n_clusters)
    visualizer = SilhouetteVisualizer(model)

    visualizer.fit(log_df_rfm)
    visualizer.poof()


# Testing clustering algorithms

## Kmeans

### StandardScaler (Not used because better results with a logarithmic scaling)

In [ ]:
for k in range(3, 5):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans_std = df_rfm.copy(deep=True)
    kmeans_std["cluster"] = kmeans.fit_predict(scaled_df_rfm)

    # Calculate the silhouette score
    silhouette = silhouette_score(scaled_df_rfm, kmeans.labels_)
    print(f"Silhouette score for {k} clusters: {silhouette}")

    print(f"Results for {k} clusters")
    benchmark_clusters(kmeans_std)
    plot_3d_clusters(kmeans_std)
    plot_radar_chart(kmeans_std)


### Scaled log

In [ ]:
for k in range(3, 6):
    kmean_log = df_rfm.copy(deep=True)
    kmeans = KMeans(n_clusters=k, random_state=42, max_iter=300)
    kmean_log["cluster"] = kmeans.fit_predict(scaled_log_df_rfm)

    # Calculate the silhouette score
    silhouette = silhouette_score(scaled_log_df_rfm, kmean_log["cluster"])
    print(f"Silhouette average score for {k} clusters: {silhouette}")

    print(f"Results for {k} clusters")
    log_df_rfm["cluster"] = kmean_log["cluster"]
    plot_3d_clusters(log_df_rfm)
    benchmark_clusters(kmean_log)
    # plot_radar_chart(log_df_rfm)


### Minmax log

In [ ]:
for k in range(3, 5):
    kmean_log = df_rfm.copy(deep=True)
    kmeans = KMeans(n_clusters=k, random_state=42, max_iter=300)
    kmean_log["cluster"] = kmeans.fit_predict(minmax_scaled_log_df_rfm)

    # Calculate the silhouette score
    silhouette = silhouette_score(minmax_scaled_log_df_rfm, kmean_log["cluster"])
    print(f"Silhouette score for {k} clusters: {silhouette}")

    print(f"Results for {k} clusters")
    plot_3d_clusters(log_df_rfm)
    log_df_rfm["cluster"] = kmean_log["cluster"]
    benchmark_clusters(kmean_log)
    plot_radar_chart(log_df_rfm)


### Log

In [ ]:
for k in range(3, 6):
    kmean_log = df_rfm.copy()
    kmeans = KMeans(n_clusters=k, random_state=42, max_iter=300)
    kmean_log["cluster"] = kmeans.fit_predict(log_df_rfm)

    # Calculate the silhouette score
    silhouette = silhouette_score(log_df_rfm, kmean_log["cluster"])
    print(f"Silhouette score for {k} clusters: {silhouette}")

    print(f"Results for {k} clusters")
    log_df_rfm["cluster"] = kmean_log["cluster"]
    benchmark_clusters(kmean_log)
    plot_3d_clusters(log_df_rfm)
    plot_radar_chart(log_df_rfm)

## DBSCAN

DBSCAN takes several hyperparameters that can influence the results.
- **eps** : maximum distance to consider two points as neighbors. It is the radius of the circle drew around the data.
- **min_samples** : Minimum points that should be in a circle of the defined radius.

There are 3 types of data points :
- **Core point** : A point is a core point if it has more than MinPts points within eps.
- **Border point** : A point which has fewer than MinPoints within eps but it is in the neighborhood of a core point.
- **Noise** : A point which is not a core point or border point.


Epsylon and min_samples help to fine-tune how DBSCAN works on the data. For this reason, we will try to evaluate the silhouette score with differents values for these two hyperparameters.

In [ ]:
params_grid = {
    "esp": [0.1, 0.2, 0.3, 0.4, 0.5],
    "min_samples": [3, 5, 7, 9, 10],
}

combinations = list(itertools.product(params_grid["esp"], params_grid["min_samples"]))

print(f"Total number of combinations: {len(combinations)}")

### Log

In [ ]:
dbscan_rfm = log_df_rfm.copy(deep=True)
dbscan_rfm = dbscan_rfm.drop(columns=["cluster"])
dbscan_rfm.head()

In [ ]:
best_params = grid_search(combinations, dbscan_rfm)

In [ ]:
print(f"Best parameters: {best_params}")

In [72]:
# Apply best params to DBSCAN
dbscan = DBSCAN(eps=best_params["epsylon"], min_samples=best_params["min_samples"])
dbscan_rfm["cluster"] = dbscan.fit_predict(dbscan_rfm)

In [ ]:
# Plot the clusters
plot_3d_clusters(dbscan_rfm, "DBSCAN")

Cluster -1 means that these instances are considered as anomalies.

In [ ]:
benchmark_clusters(df_rfm)

## AHC

A hierarchical clustering approach is based on the determination of successive clusters based on previously defined clusters. It's a technique aimed more toward grouping data into a tree of clusters called dendrograms, which graphically represents the hierarchical relationship between the underlying clusters.

The Agglomerative Hierarchical Clustering technique employs the use of distance measures to generate clusters. This generation process involves the following main steps: 
- Compute distance matrix (using distance metrics as Euclidian, Manhattan or Cosine)
- Merge closest clusters
- Update distance matrix
- Repeat previous steps until only one cluster is left

It exists different kind of hierarchical clustering :
- *Agglomerative* : Starts considering each observation as a cluster with a unique datapoint then merge iteratively clusters untils there is only one left;
- *Divisive* : Starts considering all datapoints as a unique cluster then seperates them until all datapoints become unique.

In [175]:
# select 10% of the dataset
sample = log_df_rfm.sample(frac=0.1, random_state=42)

# scaling
scaler = MinMaxScaler()
sample_scaled = scaler.fit_transform(sample)

# Shot list of distances measures methods
distance_measures = ["euclidean", "cityblock", "cosine"]
# Differents linkage methods
linkage_methods = ["average", "complete", "single", "ward", "centroid"]

In [ ]:
sample.info()

In [ ]:
sample_best_params = get_ahc_best_params(sample_scaled, distance_measures, linkage_methods)

In [172]:
sample_best_params = ("euclidean", "centroid")

In [ ]:
Z = linkage(sample_scaled, method=sample_best_params[1], metric=sample_best_params[0])

plt.figure(figsize=(20, 8))

plt.title(f'Hierarchical Clustering Dendrogram using {sample_best_params[1]} linkage method and {sample_best_params[0]} distance metric')
dendrogram(Z)

plt.xlabel('Sample indexes')
plt.ylabel('Distance')
plt.show()

In the linkage matrix Z, each row represents a merge between two clusters, and the columns contain specific information about these merges:

- Column 0: The first cluster being merged.
- Column 1: The second cluster being merged.
- Column 2: The distance between the two clusters being merged.
- Column 3: The number of original observations in the newly formed cluster.

In [ ]:
# Finding the best treshold to define number of classes
best_score = -1
max_clusters = 10

for t in np.linspace(0, np.max(Z[:, 2]), 100):
    clusters = fcluster(Z, t, criterion='distance')
    if len(set(clusters)) > 1 and len(set(clusters)) <= max_clusters:
        score = silhouette_score(sample, clusters)
        if score > best_score:
            print(score, t)
            best_score = score
            best_threshold = t

best_threshold

In [ ]:
plt.figure(figsize=(20, 10))
plt.title(
    f"Hierarchical Clustering Dendrogram using {sample_best_params[1]} linkage method and {sample_best_params[0]} distance metric"
)
dendrogram(Z, color_threshold=21)
plt.axhline(y=21, color="r", linestyle="--")
plt.xlabel("Sample indexes")
plt.ylabel("Distance")

plt.show()


In [ ]:
len(set(fcluster(Z, 21, criterion='distance')))

In [ ]:
# Full dataset
full_data_best_params = get_ahc_best_params(log_df_rfm, distance_measures, linkage_methods)


# Conclusion

Comparing our clustering algorithm, we chose to continue with K-means algoruthm. 

The hierarichal clustering is too demanding in computing performance and DBscan results gave us less precised clusters.

Let's now add few features to our RFM dataset.

In [151]:
# save df_rfm to pickle
df_rfm.to_pickle('../../data/base_df.pkl')